# 1.0 Compiled Workflow

This Jupyter notebook contains the complete Python workflow. We have also prepared 5 other Jupyter notebooks that break the workflow in this notebook into distinct parts. The code in this notebook and the broken up notebooks is identical except for code that imports/exports .csv files.

We felt that, although this notebook is useful for generating all the datasets needed for our analysis without having to switch between notebooks, it might be difficult to mark due to its length. Showing our workflow in separate, shorter, notebooks might make it easier to mark for the teaching team.

In [12]:
# !pip install pandas

In [13]:
import pandas as pd

## Declare functions

In [14]:
def is_profitable(row, threshold):
    if row['next_close'] >= (row['Close']*(threshold)):
        row['is_profitable'] = 'PROFITABLE'
    else:
        row['is_profitable'] = "UNPROFITABLE"
    return row

In [15]:
# positive = 1
# negative = -1
# neutral = 0

def determine_emotion(row):
    if row['pos_emo_y'] > row['neg_emo_y']:
        row['top_influencer_emotion'] = 1
    elif row['pos_emo_y'] < row['neg_emo_y']:
        row['top_influencer_emotion'] = -1
    else:
        row['top_influencer_emotion'] = 0
    return row

## Set parameters


In [16]:
sma_range = 10
ema_range = 10
ema_smoothing = 2

profitable_threshold = 1.02

## Import datasets

In [17]:
prices_df = pd.read_csv('../original-datasets/bitcoin.csv')
reddit_df = pd.read_csv('../original-datasets/RedditCrypto-2017.csv', header=0, names=['parent_thread_id', 'post_id', 'user_id', 'timestamp', 'pos_emo', 'neg_emo'])

In [18]:
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
prices_df.sort_values(by='Date', inplace=True)
prices_df = prices_df.reset_index(drop=True)

## 1.1 Generate labels Bitcoin prices dataset

In [19]:
prices_df['next_close'] = prices_df['Close'].shift(-1)
prices_df = prices_df.apply(is_profitable, axis=1, threshold=profitable_threshold)
prices_df.drop('next_close', axis=1, inplace=True)

## 1.2 Derive attributes for Bitcoin prices dataset

### Calculate simple moving average

In [20]:
prices_df['SMA'] = prices_df['Close'].rolling(sma_range).mean()

### Calculate exponential moving average

In [21]:
prices_df['EMA'] = prices_df['Close']
prices_df.loc[ema_range-1, 'EMA'] = (prices_df.iloc[ema_range-1])['SMA']
prices_df['EMA'] = prices_df.iloc[9:,9].ewm(span=ema_range,adjust=False).mean()

### Calulate 2-day price differences

In [22]:
prices_df['close-low'] = prices_df.apply(lambda x: x['Close']-x['Low'], axis=1)
prices_df['close-high'] = prices_df.apply(lambda x: x['Close']-x['High'], axis=1)
prices_df['SMA-close'] = prices_df.apply(lambda x: x['SMA']-x['Close'], axis=1)
prices_df['EMA-close'] = prices_df.apply(lambda x: x['EMA']-x['Close'], axis=1)


In [23]:
prices_df['prev_EMA'] = prices_df['EMA'].shift(1)
prices_df['EMA_diff'] = prices_df['EMA'] - prices_df['prev_EMA']
prices_df.drop('prev_EMA', axis=1, inplace=True)

In [24]:
prices_df['prev_SMA'] = prices_df['SMA'].shift(1)
prices_df['SMA_diff'] = prices_df['SMA'] - prices_df['prev_SMA']
prices_df.drop('prev_SMA', axis=1, inplace=True)

### Remove tuples with NA values

In [25]:
prices_df.dropna(inplace=True)

### Drop unused columns

In [26]:
prices_df.drop(['Open','High','Low','Close','Volume','Market Cap', 'SMA', 'EMA'], axis=1, inplace=True)

## 1.3 Process Reddit dataset

In [27]:
reddit_classification_df = reddit_df[['timestamp', 'pos_emo', 'neg_emo']].copy()

### Convert date column

In [28]:
reddit_classification_df['timestamp'] = pd.to_datetime(reddit_classification_df['timestamp'])
reddit_classification_df['date'] = reddit_classification_df['timestamp'].dt.date

### Aggregate sum on date column

In [29]:
reddit_classification_df = reddit_classification_df.groupby(['date'], as_index=False).sum()
reddit_classification_df['date'] = pd.to_datetime(reddit_classification_df['date'])

### Combine reddit dataframe and historical prices dataframe

In [30]:
classification_2017 = prices_df.merge(reddit_classification_df, left_on='Date', right_on='date')
classification_2017.drop('date', axis=1, inplace=True)

## 1.4 Prepare network analysis dataset

In [31]:
reddit_sna_df = reddit_df.copy()

### Remove leading and trailing white space

In [32]:
reddit_sna_df['parent_thread_id'] = reddit_sna_df['parent_thread_id'].str.strip()
reddit_sna_df['post_id'] = reddit_sna_df['post_id'].str.strip()
reddit_sna_df['user_id'] = reddit_sna_df['user_id'].str.strip()

### Extract thread id from file name

In [33]:
reddit_sna_df['parent_thread_id'] = reddit_sna_df['parent_thread_id'].str.slice(start=15, stop=21)

### Create dataframe with only "parent" posts

In [34]:
creator_posts_df = reddit_sna_df[reddit_df['post_id'] == reddit_sna_df['parent_thread_id']]
creator_posts_df = creator_posts_df.rename(columns={'user_id': 'creator_id', 'timestamp': 'thread_creation_date'})

### Join the "parent posts" dataframe with the original dataset

Match each post in the complete dataset with its corresponding "parent post"

In [35]:
joined_posts_df = creator_posts_df.merge(reddit_sna_df, left_on='post_id', right_on='parent_thread_id')
joined_posts_df.drop(['post_id_x', 'pos_emo_x', 'neg_emo_x', 'parent_thread_id_y'], axis=1, inplace=True)

### Remove tuples with the same creator_id and user_id

In [36]:
joined_posts_df = joined_posts_df[joined_posts_df['creator_id'] != joined_posts_df['user_id']]

### Remove tuples where at least one of the users involved are bots

In [37]:
joined_posts_df = joined_posts_df[joined_posts_df['creator_id'] != 'AutoModerator']
joined_posts_df = joined_posts_df[joined_posts_df['creator_id'] != 'None']
joined_posts_df = joined_posts_df[joined_posts_df['user_id'] != 'None']
joined_posts_df = joined_posts_df[joined_posts_df['user_id'] != 'AutoModerator']

### Rename columns for clarity

In [38]:
joined_posts_df = joined_posts_df.rename(
    columns={
        'parent_thread_id_x': 'parent_thread_id', 
        'post_id_y': 'post_id', 
        'timestamp': 'post_creation_date', 
        'pos_emo_y': 'pos_emo', 
        'neg_emo_y': 'neg_emo',
        'user_id': 'replier_id'
    })

### Reorder columns

In [39]:
joined_posts_df = joined_posts_df[['parent_thread_id', 'thread_creation_date', 'post_id', 'post_creation_date', 'creator_id', 'replier_id', 'pos_emo', 'neg_emo']]

In [40]:
classification_2017.to_csv('../generated-datasets/classification-dataset-with-emotion-scores.csv')
prices_df.to_csv('../generated-datasets/classification-dataset.csv')
joined_posts_df.to_csv('../generated-datasets/network-analysis-dataset.csv')

# NOTE: Please execute the KNIME workflow in `2-social-network-analysis-workflow.knwf` before continuing

## Prepare top influencer classification dataset

### Import and preprocess datasets

In [42]:
influencers_df = pd.read_csv('../generated-datasets/top-influencers.csv', usecols=['Object id']).head(3)

reddit_df = pd.read_csv('../original-datasets/RedditCrypto-2017.csv', header=0, usecols=['Source (C)', 'Source (E)', 'posemo', 'negemo'])
reddit_df = reddit_df.rename(columns={'Source (C)': 'poster', 'Source (E)':'timestamp', 'posemo': 'pos_emo', 'negemo': 'neg_emo'})
reddit_df['poster'] = reddit_df['poster'].str.strip()

prices_df = pd.read_csv('../generated-datasets/classification-dataset-with-emotion-scores.csv')

### Keep only posts by top influencers

In [43]:
reddit_df = reddit_df.merge(right=influencers_df, left_on='poster', right_on='Object id')

### Take aggreagate sum of each day

In [44]:
reddit_df['timestamp'] = pd.to_datetime(reddit_df['timestamp'])
reddit_df['timestamp'] = reddit_df['timestamp'].dt.date

reddit_df = reddit_df.groupby(['timestamp'], as_index=False).sum()
reddit_df['timestamp'] = pd.to_datetime(reddit_df['timestamp'])

### Join top influencer posts with other daily statistics

In [45]:
prices_df['Date'] = pd.to_datetime(prices_df['Date'])

reddit_df = pd.merge(prices_df, reddit_df, how='left', left_on=["Date"], right_on=["timestamp"], sort=False)

### Create column indicating the dominating emotion of the posts from the top influencers for each day
negative = -1

neutral = 0

positive = 1

In [46]:
reddit_df = reddit_df.apply(determine_emotion, axis=1)

### Drop unused columns and rename column headers

In [47]:
reddit_df = reddit_df.drop(columns=['Unnamed: 0', 'timestamp', 'pos_emo_y', 'neg_emo_y'])
reddit_df = reddit_df.rename(columns={'pos_emo_x': 'pos_emo', 'neg_emo_x': 'neg_emo'})

## Export datasets to csv

In [48]:
reddit_df.to_csv('../generated-datasets/classification-dataset-with-emotion-scores.csv')